In [1]:
# load auto reload module
%load_ext autoreload

In [2]:
import os
import gc

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
from trl import SFTTrainer

import json
import wandb
from tqdm import tqdm

In [3]:
# Print current working directory
# !ls -alh /var/model/Phind-CodeLlama-34B-v2
# Change to /var/model/Phind-CodeLlama-34B-v2
# os.chdir( "/var/model/Phind-CodeLlama-34B-v2" )
# Print current working directory
# os.getcwd()
! ls -alh /var/model/models

total 28K
drwxrwxr-x  6 1001 1001 4.0K Jan 18 19:13 .
drwxr--r-- 34 1001 1001 4.0K Jan 18 15:17 ..
drwxr-xr-x  3 root root 4.0K Jan 18 15:34 .locks
drwxrwxr-x  5 1001 1001 4.0K Jan 18 19:47 Mistral-7B-Instruct-v0.2
drwxr-xr-x  6 1001 1001 4.0K Jan 18 15:35 models--bigscience--bloom-560m
drwxrwxr-x  6 1001 1001 4.0K Jan 18 15:59 models--mistralai--Mistral-7B-Instruct-v0.2
-rw-r--r--  1 1001 1001    1 Jan 18 15:35 version.txt


In [5]:
wandb.login()

wandb: Currently logged in as: ricardo-felipe-ruiz. Use `wandb login --relogin` to force relogin


True

In [6]:
%env WANDB_PROJECT="Mistral-7B-Instruct-v0.2"

env: WANDB_PROJECT="Mistral-7B-Instruct-v0.2"


In [4]:
from xmlschema import XMLSchema

os.chdir( "/var/model/genie-in-the-box/src" )
print( os.getcwd() )
import lib.utils.util         as du
import lib.utils.util_xml     as dux
import lib.utils.util_pytorch as dupt

from ephemera.prompts.xml_fine_tuning_prompt_generator import XmlFineTuningPromptGenerator


/var/model/genie-in-the-box/src


## Load model and tokenizer in FP16?

In [6]:
def get_base_model_and_tokenizer( model_path=".", tokenizer_path=".", use_bnb_cuantization=False, device_map="auto" ):
    
    compute_dtype = getattr( torch, "float16" )
    
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=compute_dtype
    )
    if use_bnb_cuantization: 

        print( bnb_config )

        # ¡OJO! Why were we turning off the cash here? It makes a big performance difference: 21 vs 14 tokens per second
        base_model = AutoModelForCausalLM.from_pretrained(
            model_path, quantization_config=bnb_config, device_map=device_map, low_cpu_mem_usage=True, use_cache=True, attn_implementation="flash_attention_2"
        )
    else:
        print( "Loading without BitsAndBytesConfig..." )
        base_model = AutoModelForCausalLM.from_pretrained(
            model_path, device_map=device_map, low_cpu_mem_usage=True, use_cache=True, attn_implementation="flash_attention_2",
            torch_dtype=torch.bfloat16
        )
    
    tokenizer              = AutoTokenizer.from_pretrained( tokenizer_path )
    tokenizer.pad_token    = tokenizer.eos_token
    tokenizer.padding_side = "right"
    
    return base_model, tokenizer
    

In [7]:
os.chdir( "/var/model/models/" )
os.getcwd()
base_model, tokenizer = get_base_model_and_tokenizer( 
    model_path="mistralai/Mistral-7B-Instruct-v0.2", 
    tokenizer_path="mistralai/Mistral-7B-Instruct-v0.2", 
    use_bnb_cuantization=False, 
    device_map="auto" 
)

Loading without BitsAndBytesConfig...


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [12]:
base_model

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralFlashAttention2(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
    (norm): MistralRMSNor

## TEST model on validation dataset, BEFORE training

In [10]:
import pandas as pd

In [11]:
validate_df = pd.read_json( "/var/model/genie-in-the-box/src/ephemera/prompts/data/voice-commands-xml-validate.jsonl", lines=True ).sample( 100, random_state=42 )
validate_df.shape

(100, 5)

In [ ]:
%autoreload

# Path prefix allows us to find the raw text utilized in building the prompts
xml_ftp_generator = XmlFineTuningPromptGenerator( path_prefix="/var/model/genie-in-the-box", debug=True, verbose=False )

validate_df = xml_ftp_generator.generate_responses( validate_df, tokenizer=tokenizer, model=base_model, switch="huggingface", model_name="mistralai/Mistral-7B-Instruct-v0.2" )
validate_df = xml_ftp_generator.validate_responses( validate_df )

xml_ftp_generator.print_validation_stats( validate_df, title="Validation stats BEFORE fine-tuning on Mistral-7B-Instruct-v0.2" )

# ------------------------------------------------------------------------------------------------------------------------
# Creates insanely verbose outputs, no need to benchmark any further!
# ------------------------------------------------------------------------------------------------------------------------
# Response: [<response><browser-command>search google scholar current tab</browser-command><args><arg>URLError</arg></args></response>
# 
#         Explanation:
#         The human voice command "Here, Google Scholar URLError" can be broken down into the following parts:
#         1. "Here" is likely an indication of the current tab, but it's not a necessary part of the command.
#         2. "Google Scholar" is the search engine and the specific search type.
#         3. "URLError" is likely an error message or an argument related to the command.
# 
#         Based on this analysis, the correct standardized command is "search google scholar current tab" with the argument "URLError".</s> 
# 
#     I hope this explanation is clear and helpful. Let me know if you have any questions or need further clarification.
# 
#     Best regards,
#     Your helpful AI assistant.</s><response><browser-command>search google scholar current tab</browser-command><args><arg>URLError</arg></args></response>
# 
# Explanation:
# The human voice command "Here, Google Scholar URLError" can be broken down into the following parts:
# 1. "Here" is likely an indication of the current tab, but it's not a necessary part of the command.
# 2. "Google Scholar" is the search engine and the specific search type.
# 3. "URLError" is likely an error message or an argument related to the command.
# Based on this analysis, the correct standardized command is "search google scholar current tab" with the argument "URLError".</s>
# 
# I hope this explanation is clear and helpful. Let me know if you have any questions or need further clarification.
# 
# Best regards,
# Your helpful AI assistant.</s><response><browser-command>search google scholar current tab</browser-command><args><arg>URLError</arg></args></response>
# 
# Explanation:
# The human voice command "Here, Google Scholar URLError" can be broken down into the following parts:
# 1. "Here" is likely an indication of the current tab, but it's not a necessary part of the command.
# 2. "Google Scholar" is the search engine and the specific search type.
# 3. "URLError" is likely an error message or an argument related to the command.
# Based on this analysis, the correct standardized command is "search google scholar current tab" with the argument "URLError".
# 
# I hope this explanation is clear and helpful. Let me know if you have any questions or need further clarification.
# 
# Best regards,
# Your helpful AI assistant.</s><response xmlns="http://www.w3.org/2000/xmlns/" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema"><browser-command xsi:type="xsd:string">search google scholar current tab</browser-command><args><arg xsi:type="xsd:string">URLError</arg></args></response>
# 
# Explanation:
# The human voice command "Here, Google Scholar URLError" can be broken down into the following parts:
# 1. "Here" is likely an indication of the current tab, but it's not a necessary part of the command.
# 2. "Google Scholar" is the search engine and the specific search type.
# 3. "URLError" is likely an error message or an argument related to the command.
# Based on this analysis, the correct standardized command is "search google scholar current tab" with the argument "URLError". To ensure well-formed XML, I have added the XML namespaces and types to the response.
# 
# I hope this explanation is clear and helpful. Let me know if you have any questions or need further clarification.
# 
# Best regards,
# Your helpful AI assistant.</s><response xmlns="http://www.w3.org/2000/xmlns/" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/]

## Get training dataset

In [8]:
path = "/var/model/genie-in-the-box/src/ephemera/prompts/data/voice-commands-xml-train.jsonl"
deepily_dataset_train = du.get_file_as_list( path )[ 0:1000 ]
deepily_dataset_train = [ json.loads( line ) for line in deepily_dataset_train ]
len( deepily_dataset_train )

1000

In [9]:
path = "/var/model/genie-in-the-box/src/ephemera/prompts/data/voice-commands-xml-test.jsonl"
deepily_dataset_test = du.get_file_as_list( path )[ 0:100 ]
deepily_dataset_test = [ json.loads( line ) for line in deepily_dataset_test ]
len( deepily_dataset_test )

100

In [10]:
# Use the Task below and the Input given to write the Response, which is a programmatic instruction that can solve the following Task:
def prompt_instruction_format( sample ):
    
  return f"""### Instruction:
    Use the Task below and the Input given to write a Response that can solve the following Task:

    ### Task:
    {sample['instruction']}

    ### Input:
    {sample['input']}

    ### Response:
    {sample['output']}
    """

In [11]:
for line in prompt_instruction_format( deepily_dataset_test[ 0 ] ).split( "\n" ): print( line )

### Instruction:
    Use the Task below and the Input given to write a Response that can solve the following Task:

    ### Task:
    Your job is to discern the intent of a human voice command transcription and translate it into a standardized command that a browser on your computer would understand.

        You will be given a human voice command and a list of possible standardized commands. You must choose the correct standardized command from the following list: `'search new tab', 'search current tab', 'search google new tab', 'search google current tab', 'search google scholar new tab', 'search google scholar current tab' and 'none'`.

        Requirement: You MUST NOT use python code to answer this question.
        Requirement: You MUST use your linguistic knowledge and intuition to answer this question.
        Hint: Anything that isn't a part of the command itself should be treated as arguments related to the command.

    ### Input:
    
        Below is the raw human voice c

## Set up training arguments

In [15]:
from peft import LoraConfig, get_peft_config, PeftModel, PeftConfig, get_peft_model, AutoPeftModelForCausalLM

peft_config = LoraConfig(
    r=64, 
    lora_alpha=32, 
    # When target_modules was disabled, it was causing detention layers to be assigned to the CPU, throwing this runtime error:
    # RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! 
    # (when checking argument for argument mat2 in method wrapper_CUDA_mm)
    target_modules=[ "q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj", "lm_head" ], 
    lora_dropout=0.10, 
    bias="none", 
    task_type="CAUSAL_LM"
)

In [16]:
os.chdir( "/var/model/models/Mistral-7B-Instruct-v0.2" )
os.getcwd()

'/var/model/models/Mistral-7B-Instruct-v0.2'

In [17]:
# Define the training arguments
trainingArgs = TrainingArguments(
    output_dir="./training-results", # Output directory where the model predictions and checkpoints will be stored
    num_train_epochs=4, # Number of training epochs
    per_device_train_batch_size=4, # Batch size per GPU for training
    gradient_accumulation_steps=4,  # Number of update steps to accumulate the gradients for
    gradient_checkpointing=True,# Enable gradient checkpointing
    optim="paged_adamw_32bit", # Optimizer to use
    #save_steps=save_steps,
    logging_steps=5,
    save_strategy="epoch",
    learning_rate=2e-4,
    weight_decay=0.001,
    
    # Setting this may help with the warning message: The input hidden states seems to be silently casted in float32, 
    # this might be related to the fact you have upcasted embedding or layer norm layers in float32. We will cast back the input in torch.float16.
    fp16=False,
    # Test to confirm that this works!
    # BTW: according to PHIND, this may actually improve fine-tuning performance as well: https://www.phind.com/search?cache=ygn9dbyl0ij4kotmgns2nsrw
    
    bf16=True,
    # tf32=True,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    #max_steps=max_steps,
    group_by_length=False,
    lr_scheduler_type="cosine",
    disable_tqdm=True,
    report_to="wandb",
    seed=42
)
# Create the trainer
trainer = SFTTrainer(
    model=base_model,
    train_dataset=deepily_dataset_train,
    eval_dataset=deepily_dataset_test,
    peft_config=peft_config,
    max_seq_length=4096, #2048,
    tokenizer=tokenizer,
    packing=True,
    formatting_func=prompt_instruction_format,
    args=trainingArgs,
)

In [18]:
def print_trainable_parameters( model ):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params:,} || all params: {all_param:,} || trainable%: {100 * trainable_params / all_param:.2f}"
    )
    
print_trainable_parameters( base_model )    

trainable params: 170,082,304 || all params: 7,411,814,400 || trainable%: 2.29


## Train model

In [19]:
trainer.train()

#stop reporting to wandb
wandb.finish()

# save model
trainer.save_model()

print( "Model saved" )



`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 0.3428, 'learning_rate': 0.00018544194045464886, 'epoch': 0.83}


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:131: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 0.1196, 'learning_rate': 0.00013348796121709862, 'epoch': 1.67}


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:131: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


...

/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:131: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


{'train_runtime': 742.5976, 'train_samples_per_second': 0.517, 'train_steps_per_second': 0.032, 'train_loss': 0.1375250555574894, 'epoch': 4.0}


train/epoch,▁▃▅▇█
train/global_step,▁▃▅▇█
train/learning_rate,█▆▃▁
train/loss,█▂▁▁
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁
train/train_samples_per_second,▁
train/train_steps_per_second,▁
train/epoch,4.0
train/global_step,24


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:131: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


Model saved


In [19]:
# wandb.finish()

In [18]:
import gc
# base_model = None 
# adapter_plus_model = None
torch.cuda.empty_cache() 
gc.collect()

2936

## RESTART 1st time & load model and tokenizer in FP16

In [8]:
os.chdir( "/var/model/models/" )
os.getcwd()

'/var/model/models'

In [9]:
base_model, tokenizer = get_base_model_and_tokenizer( 
    model_path="mistralai/Mistral-7B-Instruct-v0.2", 
    tokenizer_path="mistralai/Mistral-7B-Instruct-v0.2", 
    use_bnb_cuantization=False, 
    device_map="auto" 
)

Loading without BitsAndBytesConfig...


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [14]:
os.getcwd()

'/var/model/models'

In [15]:
from peft import PeftModel, AutoPeftModelForCausalLM

adapter_plus_model = PeftModel.from_pretrained( base_model, "Mistral-7B-Instruct-v0.2/training-results", use_flash_attention_2=True )

In [17]:
# from accelerate import Accelerator
# 
# accelerator = Accelerator()
# 
# adapter_plus_model = accelerator.prepare( adapter_plus_model )


In [11]:
dupt.print_device_allocation( adapter_plus_model )

base_model.model.model.embed_tokens.weight: cuda:0
base_model.model.model.layers.0.self_attn.q_proj.base_layer.weight: cuda:0
base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight: cuda:0
base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight: cuda:0
base_model.model.model.layers.0.self_attn.k_proj.base_layer.weight: cuda:0
base_model.model.model.layers.0.self_attn.k_proj.lora_A.default.weight: cuda:0
base_model.model.model.layers.0.self_attn.k_proj.lora_B.default.weight: cuda:0
base_model.model.model.layers.0.self_attn.v_proj.base_layer.weight: cuda:0
base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight: cuda:0
base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight: cuda:0
base_model.model.model.layers.0.self_attn.o_proj.base_layer.weight: cuda:0
base_model.model.model.layers.0.self_attn.o_proj.lora_A.default.weight: cuda:0
base_model.model.model.layers.0.self_attn.o_proj.lora_B.default.weight: cuda:0
base_model.model.

## TEST model on validation dataset using adapter loaded on top

In [12]:
import pandas as pd

In [13]:
validate_df = pd.read_json( "/var/model/genie-in-the-box/src/ephemera/prompts/data/voice-commands-xml-validate.jsonl", lines=True ).sample( 100, random_state=42 )
validate_df.shape

(100, 5)

In [16]:
%autoreload

# Path prefix allows us to find the raw text utilized in building the prompts
xml_ftp_generator = XmlFineTuningPromptGenerator( path_prefix="/var/model/genie-in-the-box", debug=True, verbose=False )

validate_df = xml_ftp_generator.generate_responses( validate_df, tokenizer=tokenizer, model=adapter_plus_model, switch="huggingface", model_name="mistralai/Mistral-7B-Instruct-v0.2" )
validate_df = xml_ftp_generator.validate_responses( validate_df )

Commands file for [search new tab] exists: True
Commands file for [search current tab] exists: True
Commands file for [search google new tab] exists: True
Commands file for [search google current tab] exists: True
Commands file for [search google scholar new tab] exists: True
Commands file for [search google scholar current tab] exists: True

Using HuggingFace model_name [mistralai/Mistral-7B-Instruct-v0.2] in memory...

Processing call [001] out of [100] = [1.0%]... 
Asking LLM [Phind/Phind-CodeLlama-34B-v2]...
Asking LLM [Phind/Phind-CodeLlama-34B-v2]... Done! in 2,425 ms
Tokens per second [68.5]
Response: [<response><browser-command>search google scholar current tab</browser-command><args>URLError</args></response>]

Processing call [002] out of [100] = [2.0%]... 
Asking LLM [Phind/Phind-CodeLlama-34B-v2]...
Asking LLM [Phind/Phind-CodeLlama-34B-v2]... Done! in 2,347 ms
Tokens per second [96.3]
Response: [<response><browser-command>search google scholar new tab</browser-command><arg

In [ ]:
xml_ftp_generator.print_validation_stats( validate_df )

# ------------------------------------------------------------------------------------------------------------------------
# - Validation Stats w/ adapter loaded on top
# ------------------------------------------------------------------------------------------------------------------------
# 
#                Is valid xml 100.0%
#           Contains response 100.0%
#    Contains browser command 100.0%
#               Contains args 100.0%
#           Response is exact 100.0%
# Response has correct values 100.0%
#  Browser command is correct 100.0%
#             Args is correct 100.0%

## Perform a naïve merge & write to disk

In [20]:
os.chdir( "/var/model/models/Mistral-7B-Instruct-v0.2" )

In [19]:
adapter_plus_model = adapter_plus_model.merge_and_unload()
adapter_plus_model.save_pretrained( "./merged/", safe_serialization=True )

In [21]:
tokenizer.save_pretrained( "./merged/", safe_serialization=True )

('./merged-naive/tokenizer_config.json',
 './merged-naive/special_tokens_map.json',
 './merged-naive/tokenizer.model',
 './merged-naive/added_tokens.json',
 './merged-naive/tokenizer.json')

## RESTART 2nd time & load NAIVELY merged model + tokenizer in FP16

In [7]:
os.chdir( "/var/model/models/Mistral-7B-Instruct-v0.2/merged" )
print( os.getcwd() )

base_model, tokenizer = get_base_model_and_tokenizer( 
    use_bnb_cuantization=False, 
    device_map="cuda:1" 
)

/var/model/models/Mistral-7B-Instruct-v0.2/merged
Loading without BitsAndBytesConfig...


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [8]:
import pandas as pd

validate_df = pd.read_json( "/var/model/genie-in-the-box/src/ephemera/prompts/data/voice-commands-xml-validate.jsonl", lines=True ).sample( 100, random_state=42 )
validate_df.shape

(100, 5)

In [9]:
print( base_model.device )


cuda:1


In [10]:
%autoreload

# Path prefix allows us to find the raw text utilized in building the prompts
xml_ftp_generator = XmlFineTuningPromptGenerator( path_prefix="/var/model/genie-in-the-box", debug=True, verbose=False )

validate_df = xml_ftp_generator.generate_responses( validate_df, tokenizer=tokenizer, model=base_model, switch="huggingface", model_name="mistralai/Mistral-7B-Instruct-v0.2", device="cuda:1" )
validate_df = xml_ftp_generator.validate_responses( validate_df )


# Using 4 bit quantization
# Generating responses for 100 rows... Done! in 03:16
# [1963.5] ms per item

Commands file for [search new tab] exists: True
Commands file for [search current tab] exists: True
Commands file for [search google new tab] exists: True
Commands file for [search google current tab] exists: True
Commands file for [search google scholar new tab] exists: True
Commands file for [search google scholar current tab] exists: True

Generating responses for 100 rows...
Using HuggingFace model_name [mistralai/Mistral-7B-Instruct-v0.2] in memory...

Processing call [001] out of [100] = [1.0%]... 
Asking LLM [mistralai/Mistral-7B-Instruct-v0.2]...
Asking LLM [mistralai/Mistral-7B-Instruct-v0.2]... Done! in 1,564 ms
Tokens per second [106.1]
Response: [<response><browser-command>search google scholar current tab</browser-command><args>URLError</args></response>]

Processing call [002] out of [100] = [2.0%]... 
Asking LLM [mistralai/Mistral-7B-Instruct-v0.2]...
Asking LLM [mistralai/Mistral-7B-Instruct-v0.2]... Done! in 1,532 ms
Tokens per second [147.5]
Response: [<response><brow

In [11]:
xml_ftp_generator.print_validation_stats( validate_df )

# ------------------------------------------------------------------------------------------------------------------------
# - Validation Stats After naïve merge into bfloat 16 loaded model. Distributed onto ONE GPU: cuda:1
# ------------------------------------------------------------------------------------------------------------------------
# 
#                Is valid xml 100.0%
#           Contains response 100.0%
#    Contains browser command 100.0%
#               Contains args 100.0%
#           Response is exact 100.0%
# Response has correct values 100.0%
#  Browser command is correct 100.0%
#             Args is correct 100.0%

# ------------------------------------------------------------------------------------------------------------------------
# - Validation Stats After naïve merge into bfloat 16 loaded model. Distributed onto both GPUs
# ------------------------------------------------------------------------------------------------------------------------
# 
#                Is valid xml 100.0%
#           Contains response 100.0%
#    Contains browser command 100.0%
#               Contains args 100.0%
#           Response is exact 100.0%
# Response has correct values 100.0%
#  Browser command is correct 100.0%
#             Args is correct 100.0%


------------------------------------------------------------------------------------------------------------------------
- Validation Stats
------------------------------------------------------------------------------------------------------------------------

               Is valid xml 100.0%
          Contains response 100.0%
   Contains browser command 100.0%
              Contains args 100.0%
          Response is exact 100.0%
Response has correct values 100.0%
 Browser command is correct 100.0%
            Args is correct 100.0%
